In [ ]:
!git clone https://github.com/pie3636/newsjam.git
!mv newsjam/* .

Cloning into 'newsjam'...
remote: Enumerating objects: 479, done.
remote: Counting objects: 100% (479/479), done.
remote: Compressing objects: 100% (355/355), done.
remote: Total 479 (delta 259), reused 278 (delta 120), pack-reused 0
Receiving objects: 100% (479/479), 10.94 MiB | 5.58 MiB/s, done.
Resolving deltas: 100% (259/259), done.


Imports (Run Once)

In [ ]:
!pip install selenium --upgrade;
!apt install chromium-chromedriver;
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install tweepy --upgrade;
!pip install python-dotenv --upgrade;
!python -m pip install -r requirements.txt;
!python -m spacy download fr_core_news_sm;

     |████████████████████████████████| 958 kB 5.1 MB/s 
     |████████████████████████████████| 356 kB 50.6 MB/s 
     |████████████████████████████████| 138 kB 48.2 MB/s 
     |████████████████████████████████| 3.6 MB 61.1 MB/s 
     |████████████████████████████████| 55 kB 3.9 MB/s 
     |████████████████████████████████| 54 kB 3.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following addi

Twitter Setup and Authentication

In [ ]:
import nltk
nltk.download('stopwords')

import os
from dotenv import load_dotenv
from summ.lsa import LSASummarizer
from summ.bert_embed import BertEmbeddingsSummarizer as Bert
from data import scraper_functions
from classif import log_reg_classifier as lrc
import tweepy
import numpy as np
LSA = LSASummarizer()
flau = Bert('flaubert/flaubert_large_cased')
camem = Bert('camembert/camembert-large')

#this will not work if you don't have the .env file with the API keys

load_dotenv('/content/API.env')
consumer_key = os.getenv('consumer_key')
consumer_secret = os.getenv('consumer_secret')
bearer = os.getenv('bearer')
access_key = os.getenv('access_key')
access_key_secret = os.getenv('access_key_secret')

client = tweepy.Client(bearer_token=bearer, consumer_key=consumer_key, consumer_secret=consumer_secret, access_token=access_key, access_token_secret=access_key_secret)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Some weights of the model checkpoint at flaubert/flaubert_large_cased were not used when initializing FlaubertModel: ['pred_layer.proj.weight', 'pred_layer.proj.bias']
- This IS expected if you are initializing FlaubertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing FlaubertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at camembert/camembert-large were not used when initializing CamembertModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task

In [ ]:
chosen_summ = int(input("Select your summarizer: LSA (1), FlauBERT (2), or CamemBERT (3) "))
num_articles = int(input("How many articles should be selected? "))
url_list = scraper_functions.actu_autoscraper('https://actu.fr/societe/coronavirus', url_amount=num_articles)
summ_list = []
summ_url = []

if chosen_summ == 1:
  summariser = LSA
elif chosen_summ == 2:
  summariser = flau
elif chosen_summ == 3:
  summariser = camem
else:
  summariser = LSA

for x in range(3, len(url_list)):
  text, url = scraper_functions.actu_scraper(url_list[x])
  lrc_results = lrc.classifier(text)
  if lrc_results == False:
    continue
  else:
    summ_list.append(text)
    summ_url.append(url)

summ_list = [x for x in summ_list if x != '']
num_of_tweets = int(input("{} articles are appropriate for posting. How many should be tweeted? ".format(len(summ_list))))

for y in range(num_of_tweets):
  summary = summariser.get_summary(summ_list[y])
  summary = summary[0]
  print(summary)
  tweet_response = client.create_tweet(text=summary)
  client.create_tweet(text=summ_url[y], in_reply_to_tweet_id=tweet_response.data['id']) #reply to the posted tweet immediately with the original URL

Select your summarizer: LSA (1), FlauBERT (2), or CamemBERT (3) 1
How many articles should be selected? 10
7 articles are appropriate for posting. How many should be tweeted? 7
-Atlantique Batz-sur-Mer Covid : journée de dépistage par tests antigéniques lundi à Batz-sur-Mer Bon à savoir en cette période de course aux tests : une journée de dépistage est organisée ce lundi 10 janvier 2022 à Batz-sur-Mer.
Mon actu Suivre Sans surprise, malgré les vacances scolaires, la propagation rapide du variant Omicron du Covid-19 se ressent dans les écoles, les collèges et lycées de l’académie de Créteil.
Avec plus d’un million de tests réalisés par jour, les pharmacies subissent une énorme pression    Le Réveil Normand Mon actu Suivre Depuis le mois de mars 2021, les pharmaciens sont autorisés à vendre des autotests antigéniques dans leurs officines.
» A Auchan, un système (presque) calqué sur celui de Carrefour Dans le centre commercial Auchan, situé à l’espace Gramont, du côté de Balma, les autot